
### MDS - Multi-Dimensional Scaling

Non Linear dimensionality reduction methods assume that the latent variables are given by a non linear mapping of observed variables. Non linear methods of dimensionality reduction are very popular. They are much more powerful than linear ones because the relationship between latent and observed variables may be more richer than a simple matrix multiplication. These mappings are function of large number of variables and require large dataset samples and computing power. For example MDS(multidimensional scaling), ISOMAP(isometric mapping), LLE(locally linear embedding) e.t.c.

MDS means multidimensional scaling. It is not a single method but a family of methods. MDS takes a dissimilarity matrix $D$ where $D_{ij}$ represents the dissimialrity between points $i$ and $j$ and produces a mapping on lower dimension preserving the disimilarities as closely as possible. The disimilairty matrix could be observed or calculated from the given dataset.  MDS has been widely popular and developed in field of human sciences like sociology, anthropology and especially in pshycometrics.  
Let's understand it better with an example.   Table below represents correlations  between rates of different types of crimes from US in 1970. Even for a small number of variables we are unable to understand the structure of the correlation.
$\begin{array}{r|lcr}
\text{crime} & \text{Murder} & \text{Rape} & \text{Robbery} & \text{Assault} & \text{Burglary} & \text{Larceny} & \text{Auto Theft}\\
\hline
\text{Murder} & 1.00 & 0.52 & 0.34 & 0.81 & 0.28 & 0.06 & 0.11\\
\text{Rape} & 0.52 & 1.00 & 0.55 & 0.70 & 0.68 & 0.60 & 0.44\\
\text{Robbery} & 0.34 & 0.55 & 1.00 & 0.56 & 0.62 & 0.44 & 0.62\\
\text{Assault} & 0.81 & 0.70 & 0.56 & 1.00 & 0.52 & 0.32 & 0.33\\
\text{Burglary} & 0.28 & 0.68 & 0.62 & 0.52 & 1.00 & 0.80 & 0.70\\
\text{Larceny} & 0.06 & 0.60 & 0.44 & 0.32 & 0.80 & 1.00 & 0.55\\
\text{Auto Theft} & 0.11 & 0.44 & 0.62 & 0.33 & 0.70 & 0.55 & 1.00\\
\end{array}
$

Plot below represents the 2-D mappings created from the MDS.
![alt text](mds.png "")


The relative position of points on the plot depends on the disimilairality between them in the table  of correlations, i.e. crime rates which share high correlation are mapped close to each other while crime rates which do not share high correlation are mapped far. From the figure we can see that along horizontal dimension crime distribution can be interpeted as "violent crime vs property crime" whereas on vertical dimension distribution of crime can be interpeted as the "street crime vs hidden crime".



MDS can be divided into categories-  
**Metric MDS** - Metric MDS is used for the quantitative data and tries to preserve the original dissimilarity metrics.  Given a dissimialrity matrix $D$, a monotone function f, and p(number of dimension in subspace) metric MDS tries to find an optimal configuration $X \subset R^p\; s.t.\;\;\;$ $f(D_{ij})\approx d_{ij}=(x_i - x_j)^2$ . Another version of metric MDS is classical MDS(original MDS) formulation which provides closed form solution. Intead of trying to approximate the dissimilarity metrics in lower dimension, it uses eigenvalue decomposition for solution.    
**Non-Metric MDS**  - Non-metric MDS is used for ordinal data. It tries to keep the order of dissimialrity metrics intact. For example if $P_{ij}$ is dissimilarity between $i_{th}$ & $j_{th}$ and $P_{32}$ > $P_{24}$, then non-metric mds creates a mapping $s.t. \;\;d_{32} >  d_{24}$

We will implement metric MDS using SMACOF( scaling by majorization of complicated function) algorithm. Before diving into implementation of metric MDS, we need to learn a bit about MM( Majorization- Minorization) algortihm for optimization.  
**MM for finding an optima of a function**  
MM algorith is an iterative algorithm for finding optimum of a complex function.
Suppose we have a function $f(x)$ for which we need to find a minimum. Instead of directly optimizing $f(x)$ MM uses an approximating function $g(x,x_m)$ to find an optimum. If problem is to find a minimum of $f(x)$ then, $g(x,x_m)$ is called majorizing function else minorizing function and $x_m$ is called support point.  
If $g(x,x_m)$ is majorizing function of $f(x)$ then, it has to satisfy following conditions  
 1. Optimizing $g(x,x_m)$ should be easier than $f(x)$.
 2. For any $x$, $\;f(x) \; \le \;g(x,x_m)$
 3. $f(x_m) = g(x_m,x_m)$

Steps of MM algorithm 
1. choose a random support point $x_m$
2. find $x_{min}$ = $\arg\min_x {g(x,x_m)}$
3. if $f(x_{min}) - f(x_m) \approx \epsilon$ where $\epsilon$ is a very small number else go to step 4
4. set $x_m = x_{min}$ and go to step 2  
We will understand these steps better with the help of widget below. 

In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from scipy import optimize as opt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from matplotlib.pyplot import cm

x = np.array(np.arange(100))/10
y = np.abs(x-1)+np.abs(x-4)+np.abs(x-3)+np.abs(x-7)+np.abs(x-8)
pivot = 9

g_vals=[]
ax = []
ay = []
ax_ = 0
ay_ = 0
num_iter = 0
minima = 0
fun_minima = 0
g_x=0
plt.rcParams["figure.figsize"] = [12,6]
colors = cm.Reds(range(500))

def g(x):
    
    g1_ = (((x-1)**2)/(np.abs(pivot-1)) + ((x-4)**2)/(np.abs(pivot-4)) +((x-3)**2)/(np.abs(pivot-3)) + 
              ((x-7)**2)/(np.abs(pivot-7)) + ((x-8)**2)/(np.abs(pivot-8))  )
    
    g2_ = np.abs(pivot-1) + np.abs(pivot-4) + np.abs(pivot-3) + np.abs(pivot-7) + np.abs(pivot-8)
    
    g_ = 0.5*(g1_ + g2_)
    return (g_)

def calc(j):
    
    global pivot, ax, ay, g_vals,ax_, minima, fun_min, ay_,g_x
    for i in range(j):
        min_ = opt.minimize(g,x0=0)
        minima = min_.x[0]
        fun_min = min_.fun
        g_x = g(x)
        ay_ =  g(pivot)
        g_vals = g_vals + [g_x]
        ax_ = pivot
        pivot = minima
        ax = ax + [ax_]
        ay = ay + [ay_]
    
       
    
def plotter():
    
    plt.plot(x,y,"g")
    plt.ylim(10,25)
    
    
    z = int(500/((len(g_vals)+1)))
    cmap = colors[0::z]
    
    l_width = 1
    for gx,c,x_pivot,y_pivot in zip(g_vals,cmap[0:-1], ax, ay):
        
        
        plt.plot(x,gx,c=c, lw =l_width, ls="dashed")
        plt.plot(x_pivot, y_pivot,"ro")
        plt.plot([x_pivot,x_pivot],[y_pivot,0])

    plt.plot(x, g_x,  c=cmap[-1], lw=l_width+1)
    txt1 = "pivot=("+str("{0:.2f}".format(ax_))+","+str("{0:.2f}".format(ay_))+")"
    plt.annotate(txt1, xy= (ax_, ay_), xycoords="data", xytext=[ax_-2, ay_+2],
                    arrowprops=dict(arrowstyle="->",connectionstyle="arc3"),
                    bbox=dict(boxstyle="round", fc="w"), family="monospace",
                            style="normal", size="xx-large")
    
    txt1 = "minima=("+str("{0:.2f}".format(minima))+","+str("{0:.2f}".format(fun_min))+")"
    plt.annotate(txt1, xy= (minima, fun_min), xycoords="data", xytext=[minima-5, fun_min+2],
                    arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),bbox=dict(boxstyle="round", fc="w"), family="monospace",
                            style="normal",size="xx-large")
    
    
    plt.plot(ax_, ax_,"ro")
    plt.plot(minima, fun_min,"go")
    plt.plot([ax_,ax_],[ay_,0])
    plt.plot([minima, minima],[fun_min,0])
    plt.text(0,24,"Iterations = "+str(num_iter+1), horizontalalignment="left", verticalalignment="top",
            size="xx-large", family="monospace", bbox=dict(boxstyle="round", fc="w"))
    
    
def plot(t):
    global num_iter
    global num_iter
    if t>num_iter:
        calc(t-num_iter)
        plotter()
        num_iter=t
    elif t<num_iter:
        reset()
        calc(t)
        plotter()
        num_iter = t
    
    
def reset():
    global pivot, g_vals, ax, ay, ax_, ay_, minima, fun_min, num_iter 
    pivot = 9
    g_vals=[]
    ax = []
    ay = []
    ax_ = 0
    ay_ = 0
    g_x = 0
    num_iter = 0
    minima = 0
    fun_min = 0

    
interact(plot,t = widgets.IntSlider(min=1, max=12, value=1, step=1))


interactive(children=(IntSlider(value=1, description='t', max=12, min=1), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>

The plot in green is the function for which we need to find minimum. Number of iterations **t** is controlled by a widget at the top. Our initial pivot point is $x_m = 9.00$, the minimum of $g(x,9)$ is at $x_{min} = 6.49$. Now if we move the nob of widget to 2, we see that $x_{m}$ is now 6.49 and we have a new $x_{min}\;=6.20\;$ based on $g(x,6.49)$.If we increase the **t** by 1 again $x_{m}$ becomes 6.20 and $x_{min}$ value changes to 5.84. As we move the nob further towards right, we see that minima moves towards the minimum of green plot. 
So, that's how MM algorithm works. The most important part of MM algorithm is to find a good approximating function.

Now , let's move to SMACOF algorithm for metric MDS. As it was said earlier that metric MDS tries to appoximate the 
dissimilarity matrix and minimizes the stress function which is given by  

$\hspace{7em}\sigma(X) = \Sigma_{i<j}\; w_{ij}(\delta_{ij} - d_{ij}(X))^2$ , where  
$\hspace{7em} w_{ij}$ is weightage assigned to disimilarity between $i$ and $j$  
$\hspace{7em} \delta_{ij}$ is element of the given dissimilarity matrix  
$\hspace{7em} d_{ij}(X)$ is the dissimilarity from the $X$ which we need to find  
We aren't going to delve into derivation of the majorizing function for stress function. If you wish to follow please consult excellent [book](http://www.springer.com/in/book/9780387251509) (topic - Majorizing stress, page -187).  
The majorizing function for the stress function is given by $\tau(X,Z)$  
$\hspace{7em} \tau(X,Z) = \eta_{\delta}^2 + tr \;X^TVX + 2tr\; X^TB(Z)Z$  , where     
$\hspace{9em} \eta_{\delta}^2 = \Sigma_{i<j} W_{ij}\delta_{ij}^2$  
$\hspace{9em} B(Z)$ contains elements  
$\hspace{11em} b_{ij} = w_{ij}*\delta_{ij}/d_{ij} \;\; if \;d_{ij} > \;\text{0.0} \; else\; \text{0.0}$  
$\hspace{11em}  b_{ii} = - \Sigma_{j,j\ne i}\; b_{ij}$  
$\hspace{9em}  V = \Sigma_{i<j} w_{ij}*A_{ij}$  
$\hspace{11em}  A$ is a sparse matrix same size as $W$ and $s.t. A_{ii}=A_{jj}=1, A_{ji}=A_{ij}=-1$  

To implement the SMACOF algorithm, we need the derivative of $t(X,Z)$ $w.t.r$ $X$.  
$\hspace{7em} \nabla\;\tau(X,Z) = 2  XV + 2  B(Z)Z \; = \; 0$  which gives  
$\hspace{7em}VX = B(Z)Z$.  
$\hspace{7em}X = V^+B(Z)Z\;$ where $V^+$ is psuedo-inverse of $V$


Now, we have everything for the steps of SMACOF algorithm.
1. choose a random point $X_m$
2. Find the minimum of $\tau(X, X_m)$ which is given by $X_{min} = X = V^+B(Z)Z$
3. if $\sigma(X_{m}) - \sigma(X_{min}) \lt \epsilon$ break
      else set $X_m = X_{min}$ and go to step 2
      
If we don't  differentiate between dissimilarities $d_{ij}$ and $d_{jk}$, then $w_{ij}$ = 1.
Then, $V^+ = n^{-1}J$ where J is the centering matrix. Our update equation changes to  
$X_{min} = n^{-1}B(Z)(Z)$.  

Let's code the above steps of SMACOF.

In [2]:
#load the dataset
from sklearn import datasets

iris_d = datasets.load_iris()
iris = iris_d.data

In [70]:
# compute the variables which require calculation only once
from sklearn.metrics import pairwise

# distance metric calulation function
dist_mat = lambda x:pairwise.pairwise_distances(x, metric = "euclidean")

# we will denote the original distance metric with D
D = dist_mat(iris)

# stress calculation function
stress = lambda d: ((D-d)**2).sum()

def create_B(d):
    
    d[d==0.0] = np.inf
    B = D/d
    np.fill_diagonal(B, 0.0)
    
    B[range(D.shape[0]),range(D.shape[0])] = -B.sum(axis=0).T
    return(B)
    


In [ ]:

# steps of SMACOF
np.random.seed(101)
max_iter = 1000

# set lower dimenion value to 2 
p = 2
n,m = iris.shape
#choose a random pivot point
x_m = np.random.rand(n, p)

# denote the subspace distance matrix as d
d = dist_mat(x_m)

stress_old = stress(d)

tol = 1e-4
for i in range(1000):
    print(i)
    x_min = create_B(d.copy()).dot(x_m)/n
    
    d = dist_mat(x_min)
    stress_new = stress( d)
    if stress_old-stress_new < tol:
        break
    else:
        x_m = x_min
        stress_old = stress_new
    


plt.plot(x_min[:,0],x_min[:,1],"o")

The only hyperparameter required is number of dimensions in lower dimensions. If  number of dimensions is low, then solution will get distorted due to over-compression. If It is high then over-fitting will happen and solution will fit the random noise. 



### MDS in pyspark
Let's convert the python code to pyspark.

In [ ]:
from sklearn import datasets
import math as ma
import numpy as np
from pyspark.sql import types as t
from pyspark.sql import functions as f



iris_ = datasets.load_iris()

iris = iris_.data
# repartitoning the dataframe by id column will speed up the join operation 

df = spark.createDataFrame(sc.parallelize(iris.tolist()).zipWithIndex()).toDF("features", "id").repartition("id")
df.cache()

euclidean = lambda x,y:ma.sqrt(np.sum((np.array(x)-np.array(y))**2))
data_bc = sc.broadcast(df.sort("id").select("features").rdd.collect())

# create the distance metric
def pairwise_metric1(y):
    dist = []
    for x in data_bc.value:
        dist += [ma.sqrt(np.sum((np.array(x)-np.array(y))**2))]
    
    return(dist)

udf_dist1 = f.udf(pairwise_metric1, t.ArrayType(t.DoubleType()))

df = df.withColumn("D", udf_dist1("features"))
n,p = iris.shape
dim = 2
X = np.random.rand(n,dim)

# randomly initialize a solution for the pivot point.
dfrand = spark.createDataFrame(sc.parallelize(X.tolist()).zipWithIndex()).toDF("X", "id2").repartition("id2")
df = df.join(dfrand, df.id==dfrand.id2, "inner").drop("id1")

def pairwise_metric2(y):
    dist = []
    for x in X_bc.value:
        dist += [ma.sqrt(np.sum((np.array(x)-np.array(y))**2))]
    return(dist)

# create the matrix B
def B(id,x,y):
    
    y,x = np.array(y), np.array(x) 
    y[y==0.0] = np.inf
    z = -x/y
    
    z[id] = -(np.sum(z)-z[id])
    return(z.tolist())

# function for matrix multiplication using outer multiplication
def df_mult(df, col1, col2, n1, n2, matrix=True):
    
    udf_mult = f.udf(lambda x,y:np.outer(np.array(x), np.array(y)).flatten().tolist(), t.ArrayType(t.DoubleType()))
    
    df = df.withColumn("mult", udf_mult(col1, col2))
    df = df.agg(f.array([f.sum(f.col("mult")[i]) for i in range(n1*n2)])).toDF("mult")
    if not matrix:
        return(df)
    st = t.ArrayType(t.StructType([t.StructField("id",t.LongType()),t.StructField("row", t.ArrayType(t.DoubleType()))]))
    udf_arange = f.udf(lambda x:[(i,j.tolist()) for i,j in enumerate(np.array(x).reshape(n1,n2)/n1)], st)
    
    df = df.withColumn("mult", udf_arange("mult")).select(f.explode("mult").alias("mult"))
    
    df = df.select(f.col("mult.id").alias("id2"),f.col("mult.row").alias("X_min")).repartition("id2")
    return(df)

udf_B = f.udf(B, t.ArrayType(t.DoubleType()))
udf_sigma = f.udf(lambda x,y: float(np.sum((np.array(x)-np.array(y))**2)), t.DoubleType())
sigma_old = np.inf
tol = 1e-4
max_iter = 1000
for i in range(max_iter):
    X_bc = sc.broadcast(df.sort("id").select("X").rdd.collect())
    def pairwise_metric2(y):
        dist = []
        for x in X_bc.value:
            dist += [ma.sqrt(np.sum((np.array(x)-np.array(y))**2))]
        return(dist)
    udf_dist2 = f.udf(pairwise_metric2, t.ArrayType(t.DoubleType()))
    df = df.withColumn("di", udf_dist2("X"))
    
    df = df.withColumn("sigma", udf_sigma("D","di"))
    sigma_new = df.agg({"sigma":"sum"}).collect()[0][0]
    print(sigma_old, sigma_new)
    sigma_old = sigma_new
    df = df.withColumn("B", udf_B("id","D","di")).drop("di")
    
    X_min = df_mult(df, "B", "X", n, dim)
    
    df = df.join(X_min, df.id==X_min.id2).select("id", "D", f.col("X_min").alias("X"))
    # cache action will prevent recreation of dataframe from base
    df.cache()
    

Visualization of **MNIST** dataset


### Advantages and drawbacks

Metric MDs algorithm works better than classical MDS on non-linear manifolds. But, it requires $O(N^2)$ operations for processing the distance matrix. Embedding new data is hard in metric MDS. Iterative algorithms are require a lot of computing power in spark. 
